# Preliminary Models

In [3]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2 as pg

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

## Data

In [6]:
query = '''
    SELECT *
    FROM prod.features
    WHERE "Season Type" = 'NCAA Tourney' and
        holdout_s2 = 0
'''

conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

df = pd.read_sql_query(query, conn)
df_train = df[df['holdout_s1'] == 0]
df_test = df[df['holdout_s1'] == 1]

In [5]:
df_t()

,Season,DayNum,Team,Opponent,Outcome,Score,OpponentScore,NumOT,WLoc,Season Type,...,FreeThrowPct_Opponent,OffensiveRebounds_Opponent,DefensiveRebounds_Opponent,WinPct,OpponentWinPct,WinPctDiff,AvgPointsFor,AvgPointsAgainst,AvgNetPointsFor,SeedDiff
0,2003,138,1113,1242,0,76,108,0,N,NCAA Tourney,...,0.661631,14.300000,26.900000,0.620690,0.766667,-0.145977,75.9655,69.1724,6.79310,8
1,2003,137,1120,1386,1,65,63,1,N,NCAA Tourney,...,0.673550,11.172414,24.137931,0.633333,0.793103,-0.159770,70.1000,65.5667,4.53333,3
2,2003,144,1120,1393,0,78,79,0,N,NCAA Tourney,...,0.687824,14.310345,26.896552,0.633333,0.827586,-0.194253,70.1000,65.5667,4.53333,7
3,2003,137,1190,1448,0,73,76,0,N,NCAA Tourney,...,0.755330,14.758621,26.931034,0.629630,0.827586,-0.197957,79.7037,77.4815,2.22222,13
4,2003,137,1237,1246,0,64,95,0,N,NCAA Tourney,...,0.709655,12.875000,24.156250,0.593750,0.906250,-0.312500,71.2500,70.4062,0.84375,15


## Logistic Regression

## Naive Bayes

## Decision Tree

## Test Data